In [1]:
from azureml.core import Workspace
from azureml.core.authentication import AzureCliAuthentication

# cli_auth = AzureCliAuthentication()

ws = Workspace(subscription_id='738974c0-9684-498b-bbd0-c53d6d04d964',resource_group='rg-oof-cus-d-001',workspace_name='ML-OOF-CUS-D')

In [2]:
from azureml.core.dataset import Dataset

dataset2 = Dataset.get_by_name(ws, 'cleaned_dataset')

data_in2 = dataset2.as_named_input('cleaned_dataset')

In [5]:
#mlflow steps
import mlflow
# Set the MLflow experiment and start a run
mlflow.create_experiment("logging-with-mlflow pipeline221")
# mlflow_run = mlflow.start_run()


'838881032995655511'

In [6]:
#mlflow steps

experiment_name = "mlflow_spammodel_pipeline"
mlflow.set_experiment(experiment_name)


<Experiment: artifact_location='file:///c:/Users/AA706/Desktop/Mlops/mlruns/505811243535508442', creation_time=1690052827084, experiment_id='505811243535508442', last_update_time=1690052827084, lifecycle_stage='active', name='mlflow_spammodel_pipeline', tags={}>

In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

def get_aml_cluster(ws, cluster_name, vm_size='STANDARD_D2_V2', max_nodes=4):
    try:
        cluster = ComputeTarget(workspace=ws, name=cluster_name)
        print('Found existing cluster, use it.')
    except ComputeTargetException:
        config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes=max_nodes)
        cluster = ComputeTarget.create(ws, cluster_name, config)
    return cluster



# Create or get training cluster
aml_cluster = get_aml_cluster(ws, cluster_name="AA7062-com")
aml_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [8]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Environment 

aml_run_config = RunConfiguration()

# Set the target compute to your Azure Machine Learning compute cluster
aml_run_config.target = aml_cluster

# Use a curated environment
USE_CURATED_ENV = True
if USE_CURATED_ENV:

    curated_environment = Environment.get(workspace=ws, name="AzureML-Tutorial")
    aml_run_config.environment = curated_environment
else:
    # Use user-managed dependencies
    aml_run_config.environment.python.user_managed_dependencies = True

    # Add the required packages
    dependencies = CondaDependencies.create(conda_packages=['pandas', 'scikit-learn', 'seaborn', 'tqdm', 'numpy', 'tensorflow', 'spacy'],
                                            pip_packages=['azureml-sdk', 'nltk', 'string', 're'],
                                            channels=['conda-forge'])
    aml_run_config.environment.python.conda_dependencies = dependencies



In [9]:

script_params = [
    '--data', dataset2.as_named_input('cleaned_dataset'),
    '--colsample-bytree', '0.4955555555555555',
    '--learning-rate', '0.09473736842105263',
    '--max-bin', '2',
    '--max-depth', '25',
#     '--min-split_gain', '0.10526315789473684',
    '--n-estimators', '25',
    '--num-leaves', '164',
    '--reg-alpha', '0.3157894736842105',
    '--reg-lambda', '0.3157894736842105',
    '--subsample', '0.5942105263157895'
    
]

In [10]:
import azureml

azureml._restclient.snapshots_client.SNAPSHOT_MAX_SIZE_BYTES = 1000000000

AttributeError: module 'azureml._restclient' has no attribute 'snapshots_client'

In [11]:

# from azureml.pipeline.steps import PythonScriptStep

# step_2 = PythonScriptStep(name='spam model',
#                           script_name="model1_new-copy.py",
#                           source_directory="",
#                           arguments=[
#                               "--data", data_in2
#                             #   "--inputprep", processed_data,
#                               # "--output",spam_model
#                             #   "--output2",spam_pred
#                               ],
#                             inputs=[data_in2],
                            
#                             runconfig=aml_run_config,
#                             compute_target=aml_cluster


#                         #   inputs=[data_in2,processed_data],
#                         #   outputs=[spam_model,spam_pred],
#                          )


    

In [12]:
from azureml.pipeline.steps import PythonScriptStep

step_2 = PythonScriptStep(name='spam model',
                          script_name="spam_articles.py",
                          source_directory="",
                          arguments=[
                              "--data", data_in2,
                              "--colsample-bytree", "0.4955555555555555",
                              "--learning-rate", "0.09473736842105263",
                              "--max-bin", "200",
                              "--max-depth", "25",
                              "--n-estimators", "25",
                              "--num-leaves", "164",
                              "--reg-alpha", "0.3157894736842105",
                              "--reg-lambda", "0.3157894736842105",
                              "--subsample", "0.5942105263157895"
                          ],
                          inputs=[data_in2],
                                                      runconfig=aml_run_config,
                            compute_target=aml_cluster
                    
                         )


In [13]:
from azureml.pipeline.core import Pipeline

pipeline = Pipeline(ws, steps=[step_2])

In [14]:
pipeline.validate()
from azureml.core import Experiment
from azureml.widgets import RunDetails

azureml._restclient.snapshots_client.SNAPSHOT_MAX_SIZE_BYTES = 100000000000
exp = Experiment(ws, experiment_name)
run = exp.submit(pipeline)
RunDetails(run).show()

Step spam model is ready to be created [526734ca]
Created step spam model [526734ca][be76d142-5be4-488f-aeb7-bce93cc8c934], (This step will run and generate new outputs)
Submitted PipelineRun 58c00a51-b0d6-44d3-bdc1-c2da1310ffa0
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/58c00a51-b0d6-44d3-bdc1-c2da1310ffa0?wsid=/subscriptions/738974c0-9684-498b-bbd0-c53d6d04d964/resourcegroups/rg-oof-cus-d-001/workspaces/ML-OOF-CUS-D&tid=2434528d-4270-4977-81dd-a6308c1761a3


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

: 